In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import importlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch as th
import networkx as nx
import torch_geometric as pyg

import LPGNN

2022-09-05 16:05:15.940918: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-05 16:05:15.940958: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Load ASI network

In [7]:
ASI = th.Tensor(np.loadtxt('Network_Files/ASI_t0.txt')).T.type(th.int64)
ASI = ASI-1 # Convert to 0-indexing
ASI = pyg.utils.to_undirected(ASI)
ASI = pyg.data.Data(edge_index=ASI)
ASI.num_nodes = ASI.num_nodes

# Second snapshot, which we consider to be the test set.
ASI_test = th.Tensor(np.loadtxt('Network_Files/ASI_t1.txt')).T.type(th.int64)
ASI_test = ASI_test-1

# remove edges in test set which are also in training set
ASI_test_temp = th.Tensor([[-1,-1]])
for edge in ASI_test.T:
    if len(th.where((edge == ASI.edge_index.T).all(dim=1))[0]) == 0:
        ASI_test_temp = th.cat((ASI_test_temp, edge.view(1,2)), dim=0)
ASI_test = ASI_test_temp[1:].T.long()
ASI.test_pos_edge_label_index = ASI_test

In [11]:
# print ASI number of nodes, number of edges in training set, number of edges in test set, with nice formatting in three lines
print('N° of nodes:       {:,}'.format(ASI.num_nodes))
print('N° of train edges: {:,}'.format(ASI.edge_index.shape[1]/2))
print('N° of test edges:  {:,}'.format(ASI.test_pos_edge_label_index.shape[1]))

N° of nodes:       24,091
N° of train edges: 59,531.0
N° of test edges:  48,119


### Generate LaBNE ASI network. Then, get precision and recall curve by calculating the distance matrix between all nodes.

In [ ]:
ASI_LaBNE = LPGNN.labne.generateLaBNE(ASI, edge_index='edge_index', normalize_radius=False)
ASI_R_LaBNE, ASI_P_LaBNE = LPGNN.LinkPrediction.precision_recall_score_file(ASI_LaBNE, position_name='LaBNE_node_polar_positions', filename='data/Distance_Lists/ASI_LaBNE', chunk_size=200000, skip_file_prep=True, step_size=10000)